### Init our resources and catalog

In [ ]:
%run ./00-init $reset_all_data=false

### Create Volume

In [ ]:
%sql
CREATE VOLUME IF NOT EXISTS litware_unity_catalog.rag.documents_store;

In [ ]:
volume_folder =  f"/Volumes/{catalog}/{db}/documents_store"

### Ingesting PDF files as binary format using Databricks cloudFiles (Autoloader)

In [ ]:
df = (
 spark.readStream.format("cloudFiles")
  .option("cloudfiles.format", "binaryfile")
  .load(volume_folder+"/pdf_documents")
)

In [ ]:
from datetime import datetime
# Get current timestamp in a specific format (adjust as needed)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
 
# Construct the checkpoint location with the timestamp
checkpoint_location = f'dbfs:{volume_folder}/checkpoints/raw_pdfs_{timestamp}'
(
 df.writeStream.format("delta")
 .option("checkpointLocation", checkpoint_location)
 .toTable(f"{catalog}.{db}.documents_raw")
)

In [ ]:
%sql
SELECT * FROM litware_unity_catalog.rag.documents_raw

path,modificationTime,length,content
